## DB Script

---



**All corresponding DB entries are created. No need to run any script. These are for doccumentation purpose.**

1. CREATE TABLE STUDENT (     ID INT AUTO_INCREMENT PRIMARY KEY ,NAME TEXT,  USER_NAME TEXT, PASSWORD TEXT,    PUBLIC_KEY TEXT, STATUS TEXT DEFAULT 'INACTIVE');

2. CREATE TABLE CA_CERT (     ID INT AUTO_INCREMENT PRIMARY KEY ,     CSR TEXT,     CERTIFICATE TEXT, ISEXPIRED TEXT, ISREVOKED TEXT);



3. CREATE TABLE STUDENT_HISTORY (SID INT , CERT_LIST TEXT, ISEXP_LIST TEXT , EXP_ANS TEXT, EXP_COR INT, ISREV_LIST TEXT, REV_ANS TEXT, REV_COR INT);




## Generate CA Server Key Certificate

---



In [ ]:
# python functions for generating keys and certificates

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from datetime import datetime, timedelta
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes

#############################################################################
#
#    Python Function to Generate a Private Key and write to a .pem file
#
#############################################################################

def generate_private_key(filename: str, passphrase: str):
    private_key = rsa.generate_private_key(
        public_exponent=65537, key_size=2048, backend=default_backend()
    )

    utf8_pass = passphrase.encode("utf-8")
    algorithm = serialization.BestAvailableEncryption(utf8_pass)

    with open(filename, "wb") as keyfile:
        keyfile.write(
            private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=algorithm,
            )
        )

    return private_key

#############################################################################
#
#    Python Function to Generate a Public Key and write to a .pem file
#
#############################################################################

def generate_public_key(private_key, filename, validity, **kwargs):
    subject = x509.Name(
        [
            x509.NameAttribute(NameOID.COUNTRY_NAME, kwargs["country"]),
            x509.NameAttribute(
                NameOID.STATE_OR_PROVINCE_NAME, kwargs["state"]
            ),
            x509.NameAttribute(NameOID.LOCALITY_NAME, kwargs["locality"]),
            x509.NameAttribute(NameOID.ORGANIZATION_NAME, kwargs["org"]),
            x509.NameAttribute(NameOID.COMMON_NAME, kwargs["hostname"]),
        ]
    )

    # Because this is self signed, the issuer is always the subject
    issuer = subject

    # This certificate is valid from now until 90 days
    valid_from = datetime.utcnow()
    valid_to = valid_from + timedelta(days=validity)

    # Used to build the certificate
    builder = (
        x509.CertificateBuilder()
        .subject_name(subject)
        .issuer_name(issuer)
        .public_key(private_key.public_key())
        .serial_number(x509.random_serial_number())
        .not_valid_before(valid_from)
        .not_valid_after(valid_to)
    )

    # Sign the certificate with the private key
    public_key = builder.sign(
        private_key, hashes.SHA256(), default_backend()
    )

    with open(filename, "wb") as certfile:
        certfile.write(public_key.public_bytes(serialization.Encoding.PEM))

    return public_key

In [ ]:
private_key = generate_private_key("ca-private-key.pem", "secret_password")
print (private_key)

generate_public_key(private_key,
                    filename="ca-public-key.pem",
                    validity=100,
                    country="US",
                    state="Colorado",
                    locality="Fort Collins",

                    org="CS456 Trustworthy CA",
                    hostname="cs456-trustworthy-ca.com",
)

<Certificate(subject=<Name(C=US,ST=Colorado,L=Fort Collins,O=CS456 Trustworthy CA,CN=cs456-trustworthy-ca.com)>, ...)>

In [ ]:
! ls ca*
! echo
! cat ca-private-key.pem
! echo
! cat ca-public-key.pem

## Create accounts for students


---
All student info can be found here:

https://docs.google.com/spreadsheets/d/1i-hfxG_HVmhDgaaaXsNs3Zs6nBsv1LKE/edit?usp=sharing&ouid=113402613378973339442&rtpof=true&sd=true


## FLASK Server
---

In [ ]:
%%writefile ca-server.py

from cryptography import x509
from cryptography.hazmat.backends import default_backend
from getpass import getpass  # password reader
from cryptography.hazmat.primitives import serialization
from flask import Response, make_response
from cryptography.hazmat.primitives import serialization
from datetime import datetime, timedelta
from cryptography.hazmat.primitives import hashes

from flask import Flask, request, jsonify
import mysql.connector
import base64

import pymysql
import random

# Database connection configuration
configM = {
    'user': 'zarinprm',
    'password': '833289612',  # Replace with your actual password
    'host': 'faure',
    'database': 'zarinprm',  # Replace with your actual database name
    'port': 3306,
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}

app = Flask(__name__)

@app.route("/")
def get_secret_message():
    return 'Hello From The CA Server'

###################################################################################################
def generate_id(input_string):
    # Encoding the string
    encoded = base64.b64encode(input_string.encode()).decode()
    # Taking the first 5 characters
    return encoded[:5]


###################################################################################################
def get_student_by_username_and_password(username, password):
    # Establish a connection to the database
    #conn = mariadb.connect(**config)



    conn = pymysql.connect(**configM)
    print("Connected to the MariaDB database!")

    # Create a cursor object to interact with MariaDB
    cursor = conn.cursor()

    # SQL query
    query = "SELECT * FROM STUDENT WHERE USER_NAME = %s AND PASSWORD = %s"

    # Execute the query
    cursor.execute(query, (username, password))

    # Fetch one row
    row = cursor.fetchone()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    return row



def update_value_if_inactive(student_id):
    # Establish a connection to the database
    conn = pymysql.connect(**configM)
    cursor = conn.cursor()

    # SQL query
    query = """
    UPDATE STUDENT
    SET STATUS = 'ACTIVE'
    WHERE ID = %s
    """

    # Execute the query
    cursor.execute(query, (student_id))

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

@app.route('/active-account', methods=['GET'])
def retrieve():
    username = request.args.get('username')
    password = request.args.get('password')

    if not username or not password:
        return jsonify({"error": "Please provide both username and password"}), 400

    # Decoding Base64 encoded strings
    #username = base64.b64decode(encoded_username).decode('utf-8')
    #password = base64.b64decode(encoded_password).decode('utf-8')

    try:

      student = get_student_by_username_and_password(username, password)

      if student:
        #return jsonify(student)
        student_id = student['ID']
        status= student['STATUS']
        name= student['NAME']
        response = 'Hello ' + name+ '!!! . '

        if status=='INACTIVE':
            update_value_if_inactive(student_id)

            return response + 'You have successfully activated your account. Your Id is SID#'+ str(student_id)+'. Please save this for future use.'
        else:
            return response + 'Your Id is SID#'+ str(student_id)+'. Please save this for future use.'
      else:
        return jsonify({"error": "No student found with the given credentials"}), 404

    except pymysql.MySQLError as err:
      print(f"Error: {err}")
      return 'Unexpected Error Occured. Please try again sometimes later.', 404


################################################################################################

def save_student_cert(pem_data, student_id):
    conn = pymysql.connect(**configM)
    cursor = conn.cursor()

    # SQL query
    query = """
    UPDATE STUDENT
    SET PUBLIC_KEY = %s
    WHERE ID = %s
    """

    # Execute the query
    cursor.execute(query, (pem_data, student_id))

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()


@app.route('/sign-csr')
def sign_csr():

    filename = "server-public-key.pem"

    id = request.args.get('sid')

    #CSR
    csr_content = request.form.get('csr_content')
    print('csr_content')
    print(csr_content)



    # Load the CSR
    csr = x509.load_pem_x509_csr(csr_content.encode('utf-8'), default_backend())
    print(csr)

    gpass = "secret_password"
    valid_from = datetime.utcnow()
    valid_until = valid_from + timedelta(days=30)

    # Load the CA's public key
    ca_public_key_file = open("ca-public-key.pem", "rb")
    ca_public_key = x509.load_pem_x509_certificate( ca_public_key_file.read(), default_backend() )


    ca_private_key_file = open("ca-private-key.pem", "rb")
    ca_private_key = serialization.load_pem_private_key( ca_private_key_file.read(),
                                                     gpass.encode("utf-8"),
                                                     default_backend(),)

    builder = (
        x509.CertificateBuilder()
        .subject_name(csr.subject)
        .issuer_name(ca_public_key.subject)
        .public_key(csr.public_key())
        .serial_number(x509.random_serial_number())
        .not_valid_before(valid_from)
        .not_valid_after(valid_until)
    )

    for extension in csr.extensions:
        builder = builder.add_extension(extension.value, extension.critical)

    public_key = builder.sign(
        private_key=ca_private_key,
        algorithm=hashes.SHA256(),
        backend=default_backend(),
    )


    pem_data = public_key.public_bytes(serialization.Encoding.PEM)
    try:
      save_student_cert(pem_data, id)
    except pymysql.MySQLError as err:
      print(f"Error: {err}")
      return 'Unexpected Error Occured. Please try again sometimes later.' , 404

    #response = make_response(pem_data)
    #response.headers['Content-Type'] = 'application/x-pem-file'
    #response.headers['Content-Disposition'] = f'attachment; filename={filename}'

    return pem_data

##########################################################################
def get_certificate_by_id(cert_id):

    conn = pymysql.connect(**configM)
    print("Connected to the MariaDB database!")

    # Create a cursor object to interact with MariaDB
    cursor = conn.cursor()

    # SQL query
    query = "SELECT * FROM CA_CERT WHERE ID = %s"

    # Execute the query
    cursor.execute(query, (cert_id))

    # Fetch one row
    row = cursor.fetchone()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    return row


@app.route('/verify-revoked-status')
def verify_status():
    sid = request.args.get('sid')
    cert_id = request.args.get('cert_id')
    cert_id = cert_id.replace('CERT', '')

    if (not cert_id or not sid):
      return jsonify({"error": "Certificate ID and Student ID are required!!!"}), 400

    #if(not valid_sid_and_cid_pair(sid, cert_id)):
       #return jsonify({"error": "Invalid Certificate and Student Id provided!!!"}), 400


    try:
      cert = get_certificate_by_id(cert_id)

      if cert:
        status= cert['ISREVOKED']
        sts ='REVOKED'

        if status =='N':
           status = 'Not REVOKED'

        response = 'Certificate Id: ' + str(cert_id) + '. Status: '+ sts
        return response
      else:
        return jsonify({"error": "No certificate found with the given id"}), 404

    except pymysql.MySQLError as err:
      print(f"Error: {err}")
      return 'Unexpected Error Occured. Please try again sometimes later.', 404

if __name__ == "__main__":
    app.run(host="0.0.0.0", port= 10200, ssl_context=( 'server-public-key.pem', 'server-private-key.key') )  #ssl_context=('cert.crt', 'key.pem'))

Writing ca-server.py
